In [27]:
from langchain import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from fastapi import FastAPI, Request, Form, Response
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.encoders import jsonable_encoder
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
import os
import json

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.llms import Ollama

In [97]:
from litellm import completion

response = completion(
    model="ollama/zephyr", 
    messages=[{ "content": "what is cancer?","role": "user"}], 
    api_base="http://localhost:11434"
)
print(response)

ModelResponse(id='chatcmpl-95fd2076-f873-4e04-9f58-e4df3b3c611f', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Cancer is a group of diseases characterized by the uncontrolled growth and spread of abnormal cells in the body. These cells can invade and destroy nearby normal tissue, organs, and other structures, and can also spread (metastasize) to other parts of the body through the bloodstream or lymphatic system. Cancer is not a single disease but rather a complex group of diseases with many different causes, types of cells affected, and methods of progression. The exact mechanisms that initiate and promote cancer are still being studied by scientists and researchers around the world.', role='assistant'))], created=1702504097, model='ollama/zephyr', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=4, completion_tokens=110, total_tokens=114), _response_ms=4063.4750000000004)


In [ ]:
from langchain.llms import lite

In [6]:
!pip install async_generator

In [ ]:
from litellm import

In [13]:
%%writefile script.py

from litellm import completion
import litellm
async def async_ollama():
    response = await litellm.acompletion(
        model="ollama/meditron", 
        messages=[{ "content": "what is cancer?" ,"role": "user"}], 
        api_base="http://localhost:11434", 
        stream=True
    )
    async for chunk in response:
        print(chunk)

# call async_ollama
import asyncio
asyncio.run(async_ollama())


Overwriting script.py


In [11]:

%run script.py

RuntimeError: asyncio.run() cannot be called from a running event loop

In [23]:
from langchain.chat_models import ChatLiteLLM
from langchain.schema import HumanMessage

In [100]:
chat = ChatLiteLLM(
     model="ollama/zephyr", 
    api_base="http://localhost:11434"
)

In [101]:
chat

ChatLiteLLM(client=<module 'litellm' from '/opt/conda/envs/ml/lib/python3.10/site-packages/litellm/__init__.py'>, model='ollama/zephyr', openai_api_key='', azure_api_key='', anthropic_api_key='', replicate_api_key='', cohere_api_key='', openrouter_api_key='', api_base='http://localhost:11434', huggingface_api_key='', together_ai_api_key='')

In [91]:
template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
""".strip()

In [98]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

url = "http://localhost:6333"

client = QdrantClient(
    url=url, prefer_grpc=False
)

db = Qdrant(client=client, embeddings=embeddings, collection_name="vector_db")

prompt = PromptTemplate(template=template, input_variables=['context', 'question'])

retriever = db.as_retriever(search_kwargs={"k":1})

In [104]:
prompt

PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:")

In [103]:
query = "what is cancer?"
chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)
print(query)
response = qa(query)
print(response)
answer = response['result']
source_document = response['source_documents'][0].page_content
doc = response['source_documents'][0].metadata['source']
# response_data = jsonable_encoder(json.dumps({"answer": answer, "source_document": source_document, "doc": doc}))

# res = Response(response_data)

what is cancer?


> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'what is cancer?', 'result': 'Cancer is the uncontrolled growth of abnormal cells in any part of the body. These cells have lost their ability to stop dividing, leading to the formation of a mass or lump called a tumor or lesion. Cancer can arise from any organ or structure and may not be detectable until it reaches a size of 1 cm or contains 1 million cells. Exceptions include blood and bone marrow cancers, such as leukemias and lymphomas, which may not produce a mass but are evident through laboratory tests. Cancer arises from the inability of immune cells to identify and destroy these newly formed cancerous cells.', 'source_documents': [Document(page_content='What is cancer? Put simply; cancer is the abnormal growth \nof cells. Cancers arise from any organ or body structure and \nare composed of tiny cells that have lost the ability to stop \ngrowing. Occasionally , cancer may be detected “incidentall

In [38]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [54]:
def format_prompt(prompt):
    # if args.shots > 0:
    #     prompt = prompt[:-1]
    # if "orca" in args.checkpoint_name:
    #     system_msg = "You are an AI assistant who helps people find information."
    #     return f"<|im_start|> system\n{system_msg}<|im_end|>\n<|im_start|> question\n{prompt}<|im_end|>\n<|im_start|> answer\n"
    system_msg = "You are a helpful, respectful and honest assistant." + \
    "Always answer as helpfully as possible, while being safe." + \
    "Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content." + \
    "Please ensure that your responses are socially unbiased and positive in nature.\n\n" + \
    "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct." + \
    "If you don't know the answer to a question, please don't share false information."""
    return f"<|im_start|> system\n{system_msg}<|im_end|>\n <|im_start|> user\n{prompt}<|im_end|>\n <|im_start|> assistant\n"
pr = format_prompt("what is covide-19?")

In [78]:
system_msg = "You are a helpful, respectful and honest assistant." + \
"Always answer as helpfully as possible, while being safe." + \
"Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content." + \
"Please ensure that your responses are socially unbiased and positive in nature.\n\n" + \
"If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct." + \
"If you don't know the answer to a question, please don't share false information."""
prompt1 = 'what happens if listreia is left untreated?'
temp = f"<|im_start|> system\n{system_msg}<|im_end|>\n <|im_start|> user\n{prompt1}<|im_end|>\n <|im_start|> assistant\n"

In [79]:
print(temp)

<|im_start|> system
You are a helpful, respectful and honest assistant.Always answer as helpfully as possible, while being safe.Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.If you don't know the answer to a question, please don't share false information.<|im_end|>
 <|im_start|> user
what happens if listreia is left untreated?<|im_end|>
 <|im_start|> assistant



In [ ]:
template = """
<|im_start|>system
{system_msg}<|im_end|>
<|im_start|>user
{prompt}
<|im_end|>
<|im_start|>assistant""".strip()

In [63]:
prompt = PromptTemplate(template=temp, input_variables=['prompt1'])

In [68]:
prompt

PromptTemplate(input_variables=['prompt1'], template="<|im_start|> system\nYou are a helpful, respectful and honest assistant.Always answer as helpfully as possible, while being safe.Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.If you don't know the answer to a question, please don't share false information.<|im_end|>\n <|im_start|> user\n{prompt1}<|im_end|>\n <|im_start|> assistant\n")

In [64]:
llm_chain = LLMChain(prompt=prompt, llm=chat)

In [72]:
name = 'what happens if listreia is left untreated?'
lres = llm_chain.run({'prompt1':name})

In [73]:
lres

''

In [55]:
chat.generate("what is cancer?")

ValueError: Got unknown type w